# Import Uplift calculation (loads needed data)

In [1]:
%load_ext autoreload
%autoreload

from Uplift import Uplift

15:39:54 [INFO] Started loading data ...
15:39:54 [INFO] Started loading binary file ...
15:39:54 [INFO] Reading from file C:\Users\kpf\data\axinova\ax_data.feather
15:39:55 [INFO] ... finished loading binary file in 0.33s (1.11s CPU)
15:39:55 [INFO] Started loading binary file ...
15:39:55 [INFO] Reading from file C:\Users\kpf\data\axinova\ax_var_struct.feather
15:39:55 [INFO] ... finished loading binary file in 0.0s (0.02s CPU)
15:39:55 [INFO] Started loading pickle file ...
15:39:55 [INFO] Reading from file C:\Users\kpf\data\axinova\population_ratios.pkl
15:39:55 [INFO] ... finished loading pickle file in 0.0s (0.0s CPU)
15:39:55 [INFO] Started loading pickle file ...
15:39:55 [INFO] Reading from file C:\Users\kpf\data\axinova\global_code_ratios.pkl
15:39:55 [INFO] ... finished loading pickle file in 0.0s (0.0s CPU)
15:39:55 [INFO] Started loading pickle file ...
15:39:55 [INFO] Reading from file C:\Users\kpf\data\axinova\station_code_ratios.pkl
15:39:55 [INFO] ... finished loading 

# Initialize and calculate uplift

In [2]:
uplift_test = Uplift(
    name="Uplift Test",
    variables={"g_220": [0], "md_agenatrep": [2, 3]},
    stations=[], #["Aarau", "Brig"],
    time_scale="Hour",
)
print(uplift_test)

Name: 'Uplift Test'
Stations: ['Aarau', 'Basel SBB', 'Bellinzona', 'Bern', 'Biel/Bienne', 'Brig', 'Chur', 'Fribourg', 'Genève Aéroport', 'Genève Cornavin', 'Lausanne', 'Lugano', 'Luzern', 'M2', 'Neuchatel', 'Olten', 'St. Gallen', 'Winterthur', 'Zug', 'Zürich Enge', 'Zürich Flughafen', 'Zürich Flughafen - Airside', 'Zürich Flughafen - Landside', 'Zürich HB', 'Zürich Hardbrücke', 'Zürich Oerlikon', 'Zürich Stadelhofen']
Timescale: 'Hour'
Selection: 
'Anzahl Autos im Haushalt' = 'Keines'
 AND 'Alter nach 4 Kategorien' = '46-60 Jahre' OR '61+ Jahre'
Results per Variable: 
{}
Total Result: 
Empty DataFrame
Columns: []
Index: []


In [3]:
%%time
uplift_test.calculate()

Wall time: 3.91 s


# Find best slots by uplift vs. population

In [22]:
def best_pop_uplift_slots(report: Uplift, top_n: int = 20):
    result = report.result.assign(
        pop_uplift_pers=report.result["pop_uplift"] * report.result["spr"]
    )
    result = result.assign(
        pop_uplift_rank=result["pop_uplift_pers"].rank(ascending=False)
    ).query("pop_uplift_rank <= @top_n")
    return result

In [24]:
best_pop_uplift_slots(uplift_test, 30).sort_values("pop_uplift_rank")

,Station,DayOfWeek,Hour,spr,target_ratio,target_pers,target_pers_min,target_pers_max,pop_ratio,global_ratio,station_ratio,pop_uplift,global_uplift,station_uplift,pop_uplift_pers,pop_uplift_rank
3950,Zürich HB,Sunday,14,66120.484538,0.187416,12392.011297,10448.695809,14509.179791,0.079101,0.106697,0.121925,0.108315,0.080719,0.065491,7161.820483,1.0
3949,Zürich HB,Sunday,13,61105.385086,0.195175,11926.237762,10061.578877,13957.395348,0.079101,0.106697,0.121925,0.116074,0.088478,0.073250,7092.745902,2.0
540,Bern,Monday,12,37415.628752,0.198228,7416.829464,6152.750402,8805.578816,0.079101,0.106697,0.110456,0.119127,0.091531,0.087772,4457.219001,3.0
3948,Zürich HB,Sunday,12,54199.591592,0.161215,8737.783465,7317.177909,10291.002924,0.079101,0.106697,0.121925,0.082114,0.054518,0.039290,4450.546188,4.0
666,Bern,Wednesday,18,55045.621996,0.159339,8770.923295,7498.272677,10148.997427,0.079101,0.106697,0.110456,0.080238,0.052642,0.048883,4416.764239,5.0
4026,Zürich HB,Wednesday,18,79901.368588,0.133640,10677.992846,9584.539601,11834.297064,0.079101,0.106697,0.121925,0.054539,0.026943,0.011715,4357.721496,6.0
3978,Zürich HB,Thursday,18,79772.512079,0.133662,10662.536731,9540.200610,11850.954729,0.079101,0.106697,0.121925,0.054561,0.026965,0.011737,4352.458049,7.0
3953,Zürich HB,Sunday,17,76508.660808,0.134855,10317.567045,8868.654276,11881.673365,0.079101,0.106697,0.121925,0.055754,0.028158,0.012930,4265.661984,8.0
3927,Zürich HB,Saturday,15,63814.062256,0.145299,9272.103262,8015.028489,10625.907773,0.079101,0.106697,0.121925,0.066198,0.038602,0.023374,4224.352560,9.0
3929,Zürich HB,Saturday,17,70960.063317,0.137880,9783.990283,8492.332738,11172.199355,0.079101,0.106697,0.121925,0.058779,0.031183,0.015956,4170.984360,10.0
